In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [4]:
train_df_sample_1k=pd.read_csv('/kaggle/input/amex-default-prediction/train_data.csv',nrows=10000)

In [5]:
test_df_sample_1k=pd.read_csv('/kaggle/input/amex-default-prediction/test_data.csv',nrows=10000)

In [6]:
train_labels=pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')

In [7]:
train_labels.shape

(458913, 2)

In [8]:
train_labels.head(2)

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0


In [9]:
train_df_sample_1k.head(2)

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217


In [10]:
train_df_sample_1k.shape

(10000, 190)

In [11]:
#merge label and train datasets
merged_train_df=pd.merge(train_df_sample_1k,train_labels,on=['customer_ID'],how='inner')


In [12]:
merged_train_df.shape

(10000, 191)

In [13]:
merged_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 191 entries, customer_ID to target
dtypes: float64(185), int64(2), object(4)
memory usage: 14.6+ MB


In [14]:
#preprocessing dataset
#converting to objects using encoder
merged_train_df.drop(columns=['customer_ID','S_2'],axis=1,inplace=True)
test_df_sample_1k.drop(columns=['S_2'],axis=1,inplace=True)

In [15]:
#dropping columns having more NAN values
merged_train_df.isnull().sum()

P_2         64
D_39         0
B_1          0
B_2          0
R_1          0
          ... 
D_142     8428
D_143      153
D_144       63
D_145      153
target       0
Length: 189, dtype: int64

In [16]:
for cols in merged_train_df.columns:
    if (merged_train_df[cols].isnull().sum())/((merged_train_df.shape[0])*100) >= 75:
        merged_train_df.drop(columns=[cols],axis=1,inplace=True)
        
        

In [17]:
for cols in test_df_sample_1k.columns:
    if (test_df_sample_1k[cols].isnull().sum())/((test_df_sample_1k.shape[0])*100) >= 75:
        test_df_sample_1k.drop(columns=[cols],axis=1,inplace=True)

In [18]:
test_df_sample_1k.shape

(10000, 189)

In [19]:
#convert all categorical to string
train_df = merged_train_df.astype({"B_30": 'str', "B_38": 'str'})

test_df = test_df_sample_1k.astype({"B_30": 'str', "B_38": 'str'})

train_df = train_df.astype({"D_114": 'str', "D_116": 'str', "D_117": 'str', "D_120": 'str', "D_126": 'str', "D_68": 'str'})

test_df = test_df.astype({"D_114": 'str', "D_116": 'str', "D_117": 'str', "D_120": 'str', "D_126": 'str', "D_68": 'str'})

In [20]:
X=train_df.drop(columns='target')
y=train_df['target']

In [21]:
categorical = list(X.select_dtypes('object').columns)
numerical=list(X.select_dtypes('number').columns)

In [22]:
categorical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent',missing_values=np.nan)),
    ('encoder',OneHotEncoder(handle_unknown='ignore',sparse=False)),
    ('scaler',StandardScaler())
])

print(categorical_pipeline)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('scaler', StandardScaler())])


In [23]:
numerical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent',missing_values=np.nan)),
    ('scaler',StandardScaler())
])
print(numerical_pipeline)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())])


In [24]:
pre_process=ColumnTransformer([
    ('cat',categorical_pipeline,categorical),
    ('num',numerical_pipeline,numerical)
])
print(pre_process)

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['D_63', 'D_64', 'D_68', 'B_30', 'B_38',
                                  'D_114', 'D_116', 'D_117', 'D_120',
                                  'D_126']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scaler', StandardScaler())]),
                                 

In [25]:
#split train and test data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)


In [26]:
y_test.shape

(3000,)

In [27]:
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.metrics import accuracy_score,r2_score

In [28]:
lgb_model = lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0)
rf_model=RandomForestClassifier(n_estimators = 10, random_state = 42, n_jobs = -1)

In [29]:
steps = [
        ('preprocess', pre_process),
        ('model_estimator', RandomForestClassifier(random_state = 42))
    ]
pipe = Pipeline(steps, verbose=True)

In [30]:
pipe.set_params(model_estimator=lgb.LGBMClassifier())
model=pipe.fit(X_train,y_train)

[Pipeline] ........ (step 1 of 2) Processing preprocess, total=   0.2s
[Pipeline] ... (step 2 of 2) Processing model_estimator, total=   1.5s


In [31]:
y_preds=pd.DataFrame(pipe.predict(X_test), columns=['prediction'])

In [32]:
pipe.score(X_train,y_train)

1.0

In [33]:
accuracy_score(y_preds,y_test)

0.986

In [34]:
y_test_pred = pipe.predict(test_df_sample_1k)

In [35]:
# test_all=pd.read_csv('/kaggle/input/amex-default-prediction/test_data.csv')

In [36]:
output_1=pipe.predict(test_df_sample_1k)

In [37]:
output = pd.DataFrame({'customer_ID': test_df_sample_1k.index,'prediction': output_1})
output.to_csv('submission.csv', index=False, header=True)